[1] 모듈 로딩 및 데이터 준비

In [1]:
# 모듈 로딩

from Korpora import Korpora     # Open Korean Dataset
from konlpy.tag import *        # 형태소 분석기
import spacy                    # 형태소 분석기
from torch.utils.data import Dataset, DataLoader # Pytorch Dataset관련 모듈
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize, wordpunct_tokenize

In [2]:
## 데이터 로딩
nsmc = Korpora.load("nsmc")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\KDP-23\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\KD

In [3]:
## test  데이터셋을  DataFrame으로 로딩
nsmcDF = pd.DataFrame(nsmc.test)
nsmcDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    50000 non-null  object
 1   label   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [4]:
## 단어 사전을 생성 시 활용
class TextDataset(Dataset):
    def __init__(self, feature, label):
        self.feature = feature
        self.label = label
        self.length = feature.shape[0]

    def __len__(self):
        return self.length
    
    def __getitem__(self, index):
        return self.feature.iloc[index], self.label.iloc[index]

In [5]:
## NSMC의 데이터셋 인스턴스 생성
nsmcDS = TextDataset(nsmcDF['text'], nsmcDF['label'])

In [6]:
for f, l in nsmcDS:
    print(f,l)
    break

굳 ㅋ 1


[2] 데이터 전처리 

- 대소문자 통일

In [10]:
# list 컴프레이션 사용
all_text = (text.lower() for text in nsmcDS) 

- 토큰화

In [9]:
all_tokens=[]
for text in all_text:
    all_tokens.append(wordpunct_tokenize(text))

- 불용어 & 구두점 제거

In [10]:
## 불용어 추출
from nltk.corpus import stopwords
import string

eng_stopwords = stopwords.words('english')
pun = list(string.punctuation)

In [16]:
# 불용어, 구두점, 길이 짧은 토큰 삭제
for tokenList in all_tokens:
    for token in tokenList[::-1]: # 뒤에서 돌린다는게 뭔말인지 아직잘 모르겠음
        if token.isnumeric(): # 숫자를 먼저 빼내기위해서 앞에오고 숫자라면 제거 아니면 다음 elif으로 넘어가 조건문으로 들어간다
            tokenList.remove(token)
        
        elif (token in eng_stopwords) or (token in pun) or (len(token) <= 2):
            tokenList.remove(token)
    

In [17]:
for tokenList in all_tokens:
    print(tokenList)

- 단어별 빈도수 계산

In [18]:
# 토큰을 키로 해서 빈도수 저장
token_freqs = {}

# 라인(줄)별 토큰을 읽어서 빈도 체크
for tokenList in all_tokens:
    for token in tokenList:
        # 토튼 키가 존재하지 않으면 키로 추가 후 1로 설정
        if token not in token_freqs:
            token_freqs[token] = 1
        # 이미 존재하는 토큰 키이면 값을 1 증가
        else:
            token_freqs[token] += 1


In [ ]:
# data = {'a': 1 , 'a': 2}
# 'a' in data

token_freqs

In [ ]:
### 빈도별로 몇 개의 단어가 존재하는지 체크 
token_freqs.items() 

In [ ]:
# 많이 나온데로 정렬하기 위해
## 빈도수별로 단어 또는 단어 개수 저장
freqsDict = {}
for k, v in token_freqs.items(): # v를 리스트로??
    if v not in freqsDict:
        # freqsDict[v]= [k] # 단어를 가져갈거면
        # freqsDict[v]=1 
        freqsDict[v]=[1, [k]]
        
    else : 
        # freqsDict[v].append(k) # 개수로 가져갈려면
        # freqsDict[v]+=1
        freqsDict[v][1].append(k)
        freqsDict[v][0] += 1
        
print(freqsDict)

### 빈도수를 보고 이 단어를 지울건지 살릴건지 결정할라고

In [ ]:
sorted(freqsDict.items(), reverse=True)

- [3] 단어 집합 / 단어 사전 생성

In [23]:
# 빈도가 높게 나오는 순서대로 단어 정렬 => 값을 기준으로 정렬
#                   (k, y) --------------  v >>> v로 정렬해줘
sortedTokens = sorted(token_freqs.items(), key=lambda x:x[1], reverse=True)

In [24]:
## 단어사전 생성 및 초기화
## 특수 토큰 : 'PAD' , 'OOV' 또는 'UNK'
PAD_TOKEN , OOV_TOKEN = 'PAD','OOV'

vocab = {PAD_TOKEN:0 , OOV_TOKEN :1}

In [ ]:
## 데이터 코퍼스에서 추출된 토큰(단어)들
for idx, tk in enumerate(sortedTokens, 2):
    vocab[tk[0]] = idx
print(vocab) 
# 'PAD'[패딩에 사용한 0]: 0, 'OOV'[인코딩안에 없는 단어 인코딩]: 1 
                            # OOV 가 많이 있다면 잘못된 단어 사전이라는 것ㅕㅕㅕㅓㅕㅓ을 알수있다

In [26]:
## 텍스트 문장 ==> 수치화[인코딩] 
encodingData = []
for tokenList in all_tokens:
    # 1개 문장 인코딩
    sent = []
    print(f' 문장 {tokenList}')
    for token in tokenList:
        sent.append(vocab[token])
    # 인코딩 된 문장 저장
    encodingData.append(sent)
    print(f' ==> 인코딩 {sent}\n')

- [4] 패딩 (Padding) <hr>

In [27]:
# 문장들 마다 길이 체크 >> 문장의 길이를 맞춰줄 계획
dataLen = list (len(sent) for sent in encodingData )
# dataLen = [len(sent) for sent in encodingData] 

In [ ]:
# 히스토그램을보고 문장 길이 확인
import matplotlib.pyplot as plt
plt.hist(dataLen, bins=5)
plt.show()

In [ ]:
## 기준 길이
MAXLen = max(dataLen)
MAXLen

In [53]:
## [1] 가장 긴 문장 길이 기준
for idx, sent in enumerate(encodingData):
    cur_len = len(sent)
    if cur_len < MAXLen:
        # print(cur_len, MAXLen - cur_len ) # 확인용
        encodingData[idx] = sent + [0]*(MAXLen - cur_len)

In [ ]:
for _ in encodingData[:3]:
    print(len(_), _)

In [61]:
## [2] 지정한 문장 길이 기준

MAXLen = 100

for idx, sent in enumerate(encodingData):
    cur_len = len(sent)
    if cur_len < MAXLen:
        encodingData[idx] = sent + ([0]*(MAXLen - cur_len))
    else:
        # 앞부분 자를 때
        encodingData[idx] = sent[cur_len - MAXLen:]

        # # 뒷부분 제거 시
        # encodingData[idx] = sent[:MAXLen]

In [ ]:
maxLen = 8
a= [1,2,3,4,5,6,7,8,9,0]

# 필요한 부분 a = [1,2,3,4,5,6,7,8] : 뒷부분 자르기
b = a[:maxLen]

# 필요한 부분 a = [3,4,5,6,7,8,9,0]] : 앞부분 자르기
c = a[len(a) - maxLen:]

b, c